In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm


1.1 Get the list of animes

With the for loop,we get the url of all the pages (50 per page except the last one which has less, 383 pages).

In [3]:
urls=[]
for i in tqdm(range(0,20000,50)):
  url='https://myanimelist.net/topanime.php?limit='+str(i)
  
  soup= BeautifulSoup(requests.get(url).text,'html.parser')

  for tag in soup.find_all('tr'):
    links=tag.find_all('a')
    for l in links:
      if type(l.get('id'))== str and len(l.contents[0]) >1:
        urls.append(l.get('href'))

100%|██████████| 400/400 [03:22<00:00,  1.97it/s]


In [4]:
len(urls)

19153

here we saved urls as .txt

In [5]:
with open('urls.txt', 'w', encoding='utf-8') as f: 
    for line in urls:
        f.write(line)
        f.write('\n')

1.2 Crawl animes

Create the folders. Put all of them under "pages". Each folder has a name that refers to the number of the page from which the links it contains come from.

Here we used MyDriver to collect anime folders by pages.

In [9]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [12]:
import os
for page in tqdm(range(1, 384)):
    folder = "page"+str(page)
    path = "/content/drive/MyDrive/Anime_folder"+folder
    os.mkdir(path)

  0%|          | 0/383 [00:00<?, ?it/s]


FileExistsError: ignored

In [13]:
!rm -rf pages
   #to delete folders that are not needed.

Try for the first 150:ok. Try to collect them in group of 3.

In [15]:
for page in tqdm(range(4, 8)):  # page 1 --> 383
    folder = "/content/drive/MyDrive/Anime_folderpage"+str(page+1)
    update_page = 50*page
    for i in range(0,50):   # 1 -> 50
        url = f'{urls[update_page+i]}'
        response = requests.get(url)   
        filename = r""+folder+"/anime_"+str(update_page+i+1)+".html"
        with open(filename,'w', encoding='utf-8') as f:
            f.write(response.text)

100%|██████████| 4/4 [02:52<00:00, 43.11s/it]



1.3 Parsing downloaded pages.

Here is created functions in order to get needed information, like the title or the number of series and so on. To do that we send the url of the anime to the function and we extract needed info.


In [16]:
def AnimeTitle(html):
  with open(html,'r') as f:
    soup= BeautifulSoup(f)
    animeTitle=soup.find("h1", attrs = {"class": "title-name h1_bold_none"}).string
    return(animeTitle)

In [22]:
def animeScore(html):
  with open(html,'r') as f:
    soup= BeautifulSoup(f)
    for i in range(10):
      score=soup.find_all('div' ,attrs = {"class": "score-label score-"+str(i)})
      for j in score:
        return float(j.text)

In [27]:
def AnimeRank(html):
  with open(html,'r') as f:
    soup= BeautifulSoup(f)
    rank=int(str(soup.find(class_="numbers ranked").text).split('#')[-1])  
    return(rank)

In [31]:
def animeNumberofepisode(html_string):
  with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    
    A=soup.find(text=re.compile('Episodes:')).parent.parent.text.split()[-1]      
    print(A)

In [33]:
def AnimePopularity(html):
  with open(html,'r') as f:
    soup= BeautifulSoup(f)
    rank=int(str(soup.find(class_="numbers popularity").text).split('#')[-1])  
    return(rank)

In [35]:
def animeDescription(html_string): 
  with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    A=soup.find(itemprop="description")
    return A.get_text()

In [37]:
def animeUsers(html_string): 
  with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    A=soup.find(itemprop="ratingCount")
    return int(A.get_text())

In [39]:
def animeType(html_string):
  with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    
    A=soup.find(text=re.compile('Type:')).parent.parent.text.split()[-1]      
    print(A)

In [25]:
AnimeTitle('/content/drive/MyDrive/Anime_folderpage1/anime_10.html')

'Gintama: The Final'

In [26]:
animeScore('/content/drive/MyDrive/Anime_folderpage1/anime_13.html')

8.97

In [28]:
AnimeRank('/content/drive/MyDrive/Anime_folderpage1/anime_13.html')

13

In [30]:
import re

In [32]:
animeNumberofepisode('/content/drive/MyDrive/Anime_folderpage1/anime_1.html')

64


In [34]:
AnimePopularity('/content/drive/MyDrive/Anime_folderpage1/anime_13.html')

23

In [36]:
animeDescription('/content/drive/MyDrive/Anime_folderpage1/anime_10.html')

'New Gintama movie.'

In [38]:
animeUsers('/content/drive/MyDrive/Anime_folderpage1/anime_13.html')

1227199

In [40]:
animeType('/content/drive/MyDrive/Anime_folderpage1/anime_11.html')

TV


In [43]:
def fai_date(a: list):
    
    string = ''
    for i in a:
        string += i
        if i != a[-1]:
            string += ' '
    string = string.replace(',', '')
    date = datetime.strptime(string, '%b %d %Y')
    return date

In [41]:
def animeRelDate(html_string):
  with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    
    A= fai_date(soup.find(text=re.compile('Aired:'), class_="dark_text").parent.text.split()[1:4]  )   
    print(A)

In [45]:
def animeEndDate(html_string):
  with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    
    A= fai_date(soup.find(text=re.compile('Aired:'), class_="dark_text").parent.text.split()[5:8]   )  
    print(A)

In [47]:
def animeRelated(html_string):  
  animeRelated = []
  with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    related = soup.find_all("table", {"class":"anime_detail_related_anime"})
    for i in related:
      links = i.find_all('a')
      for link in links:        
          animeRelated.append(f'{link.contents[0]}')

  return animeRelated

In [48]:
animeRelated('/content/drive/MyDrive/Anime_folderpage1/anime_11.html')

['Gintama', 'Gintama°', 'Gintama.: Porori-hen']

In [49]:
def animeCharacter(html_string):
  personaggi=[]
  with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    
    A= soup.find_all(class_="h3_characters_voice_actors")    
    for a in A:
      personaggi.append(a.text)
    return(personaggi)

In [50]:
animeCharacter('/content/drive/MyDrive/Anime_folderpage1/anime_11.html')

['Sakata, Gintoki',
 'Kagura',
 'Katsura, Kotarou',
 'Takasugi, Shinsuke',
 'Shimura, Shinpachi',
 'Kamui',
 'Elizabeth',
 'Imai, Nobume',
 'Sadaharu',
 'Sakamoto, Tatsuma']

In [51]:
def animeVoices(html_string):
  personaggi=[]
  with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    
    A= soup.find_all(class_="va-t ar pl4 pr4")   
    for a in A:
      personaggi.append(a.text)

      
    return(personaggi)

In [52]:
animeVoices('/content/drive/MyDrive/Anime_folderpage1/anime_11.html')

['\nSugita, Tomokazu\nJapanese\n',
 '\nKugimiya, Rie\nJapanese\n',
 '\nIshida, Akira\nJapanese\n',
 '\nKoyasu, Takehito\nJapanese\n',
 '\nSakaguchi, Daisuke\nJapanese\n',
 '\nHino, Satoshi\nJapanese\n',
 '\nHirano, Aya\nJapanese\n',
 '\nTakahashi, Mikako\nJapanese\n',
 '\nMiki, Shinichiro\nJapanese\n']

In [53]:
def createstaff(ll):
    listto = []
    for i in ll:
        if i['href'].startswith('https://myanimelist.net/people/') and i['href'] not in listto:
            if i.text != '\n\n':
                j = str(i.text).replace('\n', '')
                j = re.sub(' +', ' ', j)
            if j[0] == ' ':
                j = j[1:]
            if j[-1] == ' ':
                j = j[:-1]
            listto.append(j)
    return listto      

In [57]:
def animeStaff(html_string):
   with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    
    A=  createstaff(soup.find_all('a', href=True))     
    print(A)

In [54]:
animeStaff('/content/drive/MyDrive/Anime_folderpage1/anime_11.html')

['Sugita, Tomokazu', 'Sugita, Tomokazu', 'Kugimiya, Rie', 'Kugimiya, Rie', 'Ishida, Akira', 'Ishida, Akira', 'Koyasu, Takehito', 'Koyasu, Takehito', 'Sakaguchi, Daisuke', 'Sakaguchi, Daisuke', 'Hino, Satoshi', 'Hino, Satoshi', 'Hirano, Aya', 'Hirano, Aya', 'Takahashi, Mikako', 'Takahashi, Mikako', 'Miki, Shinichiro', 'Miki, Shinichiro', 'Miki, Shinichiro', 'Fujita, Youichi', 'Fujita, Youichi', 'Miyawaki, Chizuru', 'Miyawaki, Chizuru', 'Takamatsu, Shinji', 'Takamatsu, Shinji', 'Kobayashi, Katsuyoshi']
